In [21]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
images_path = "../data/images"
masks_path = "../data/masks"
output_images_path = "../data/processed_data/images"
output_masks_path = "../data/processed_data/masks"

os.makedirs(output_images_path, exist_ok=True)
os.makedirs(output_masks_path, exist_ok=True)


TARGET_SIZE = (256, 256)

In [3]:
feature_path = "../data/processed_data/features"
mask_path = "../data/processed_data/masks"

In [4]:
def load_data(feature_path, mask_path):
    X, Y = [], []
    feature_files = sorted([f for f in os.listdir(feature_path) if f.endswith('.npy')])
    mask_files = sorted([f for f in os.listdir(mask_path) if f.endswith('.npy')])

    for f_file, m_file in zip(feature_files, mask_files):

        features = np.load(os.path.join(feature_path, f_file))
        mask = np.load(os.path.join(mask_path, m_file))

        X.append(features) 
        Y.append(mask)      

    return np.array(X), np.array(Y)

In [5]:
X_train, Y_train = load_data(feature_path, mask_path)

In [6]:
def build_unet_decoder(input_shape=(8, 8, 2048)):
    inputs = layers.Input(shape=input_shape)


    x = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding="same")(inputs)  
    x = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(x)
    x = layers.Conv2D(512, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding="same")(x)  
    x = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(x)
    x = layers.Conv2D(256, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding="same")(x) 
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding="same")(x) 
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)

    x = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding="same")(x)  
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)

   
    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid")(x)

    
    model = Model(inputs=inputs, outputs=outputs)
    return model

model = build_unet_decoder()
model.summary()


2024-12-15 16:05:13.421682: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-12-15 16:05:13.421721: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-15 16:05:13.421726: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-15 16:05:13.421749: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-15 16:05:13.421765: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 8, 8, 2048)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 16, 16, 512)    │     4,194,816 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 32, 32, 256)    │       524,544 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 64, 64, 128)    │       131,200 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_3              │ (None, 128, 128, 64)   │        32,832 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_4              │ (None, 256, 256, 32)   │         8,224 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 256, 256, 1)    │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,178,945 (42.64 MB)

 Trainable params: 11,178,945 (42.64 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:

X_train = np.squeeze(X_train)  


Y_train = np.expand_dims(Y_train, axis=-1)  


print(f"Feature shape: {X_train.shape}, Mask shape: {Y_train.shape}")


Feature shape: (2594, 8, 8, 2048), Mask shape: (2594, 256, 256, 1)


In [8]:

unique_values = np.unique(Y_train)
print(f"Maskelerdeki benzersiz değerler: {unique_values}")

Maskelerdeki benzersiz değerler: [  0 255]


In [9]:

if unique_values.max() > 1:
    Y_train = Y_train / 255.0
    print("Maskeler normalize edildi: Değerler 0-1 aralığına getirildi.")


print(f"Yeni mask değerleri: {np.unique(Y_train)}")

Maskeler normalize edildi: Değerler 0-1 aralığına getirildi.
Yeni mask değerleri: [0. 1.]


In [11]:

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss="binary_crossentropy",
              metrics=["accuracy"])

callbacks = [
    EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    ModelCheckpoint("../models/unet_resnet50_best.keras", save_best_only=True)
]

In [12]:

history = model.fit(
    X_train, Y_train,
    validation_split=0.2,
    batch_size=8,
    epochs=30,
    callbacks=callbacks
)

model.save("../models/unet_resnet50_final.keras")
print("Model eğitimi tamamlandı ve kaydedildi.")

Epoch 1/30


2024-12-15 16:07:28.983217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


260/260 ━━━━━━━━━━━━━━━━━━━━ 57s 199ms/step - accuracy: 0.8190 - loss: 0.3906 - val_accuracy: 0.9001 - val_loss: 0.3386
Epoch 2/30
260/260 ━━━━━━━━━━━━━━━━━━━━ 51s 196ms/step - accuracy: 0.9291 - loss: 0.2278 - val_accuracy: 0.9169 - val_loss: 0.2173
Epoch 3/30
260/260 ━━━━━━━━━━━━━━━━━━━━ 51s 195ms/step - accuracy: 0.9438 - loss: 0.1388 - val_accuracy: 0.9166 - val_loss: 0.2145
Epoch 4/30
260/260 ━━━━━━━━━━━━━━━━━━━━ 50s 194ms/step - accuracy: 0.9505 - loss: 0.1208 - val_accuracy: 0.9213 - val_loss: 0.2111
Epoch 5/30
260/260 ━━━━━━━━━━━━━━━━━━━━ 50s 192ms/step - accuracy: 0.9577 - loss: 0.1009 - val_accuracy: 0.9248 - val_loss: 0.2189
Epoch 6/30
260/260 ━━━━━━━━━━━━━━━━━━━━ 50s 192ms/step - accuracy: 0.9645 - loss: 0.0844 - val_accuracy: 0.9051 - val_loss: 0.2432
Epoch 7/30
260/260 ━━━━━━━━━━━━━━━━━━━━ 50s 192ms/step - accuracy: 0.9690 - loss: 0.0753 - val_accuracy: 0.9214 - val_loss: 0.2443
Epoch 8/30
260/260 ━━━━━━━━━━━━━━━━━━━━ 50s 192ms/step - accuracy: 0.9752 - loss: 0.0585 - val

DATA AUGMENTATION ILE EGITILEN MODEL

In [26]:
data_gen_args = dict(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.1,
    fill_mode="nearest"
)

In [27]:
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

In [28]:
def data_generator(features, masks, batch_size=8):
    seed = 42 
    image_generator = image_datagen.flow(features, batch_size=batch_size, seed=seed)
    mask_generator = mask_datagen.flow(masks, batch_size=batch_size, seed=seed)
    for img, mask in zip(image_generator, mask_generator):
        yield img, mask

In [29]:
batch_size = 8
train_generator = data_generator(X_train, Y_train, batch_size)

In [30]:
callbacks = [
    EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    ModelCheckpoint("../models/unet_resnet50_augmented_best.keras", save_best_only=True)
]

In [31]:
X_train_split, X_val_split, Y_train_split, Y_val_split = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=42
)

In [32]:
train_generator = data_generator(X_train_split, Y_train_split, batch_size)

val_generator = data_generator(X_val_split, Y_val_split, batch_size)

In [25]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train_split) // batch_size,
    validation_data=val_generator,
    validation_steps=len(X_val_split) // batch_size,
    epochs=30,
    callbacks=callbacks
)

/opt/anaconda3/envs/tensorflow-metal/lib/python3.9/site-packages/keras/src/legacy/preprocessing/image.py:619: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (2075, 8, 8, 2048) (2048 channels).
  warnings.warn(


Epoch 1/30
259/259 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.9229 - loss: 0.1766

/opt/anaconda3/envs/tensorflow-metal/lib/python3.9/site-packages/keras/src/legacy/preprocessing/image.py:619: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (519, 8, 8, 2048) (2048 channels).
  warnings.warn(


259/259 ━━━━━━━━━━━━━━━━━━━━ 123s 474ms/step - accuracy: 0.9229 - loss: 0.1766 - val_accuracy: 0.9227 - val_loss: 0.1747
Epoch 2/30
259/259 ━━━━━━━━━━━━━━━━━━━━ 121s 468ms/step - accuracy: 0.9308 - loss: 0.1593 - val_accuracy: 0.9272 - val_loss: 0.1869
Epoch 3/30
259/259 ━━━━━━━━━━━━━━━━━━━━ 124s 478ms/step - accuracy: 0.9330 - loss: 0.1583 - val_accuracy: 0.9261 - val_loss: 0.1796
Epoch 4/30
259/259 ━━━━━━━━━━━━━━━━━━━━ 127s 491ms/step - accuracy: 0.9304 - loss: 0.1656 - val_accuracy: 0.9208 - val_loss: 0.2072
Epoch 5/30
259/259 ━━━━━━━━━━━━━━━━━━━━ 260s 1s/step - accuracy: 0.9327 - loss: 0.1593 - val_accuracy: 0.9077 - val_loss: 0.2519
Epoch 6/30
259/259 ━━━━━━━━━━━━━━━━━━━━ 146s 564ms/step - accuracy: 0.9279 - loss: 0.1812 - val_accuracy: 0.8900 - val_loss: 0.3626
